In [1]:
import pandas as pd
import numpy as np 


In [2]:
# importing all of our original csv files as seperate dataframes
games = pd.read_csv('../Data/games.csv'); 
players = pd.read_csv('../Data/players.csv'); 
plays = pd.read_csv('../Data/plays.csv'); 

combined_df = []; 
final_df = []; 

In [3]:
# join all the datasets together into a unified dataset for manipulation
# WARNING: DISCOVERED THAT THE DATASET ONLY CONTAINS 12,486 PLAYS FOR 136 GAMES PLAYED IN THE 2022 SEASON 
# THE LARGEST AMOUNT OF DATA IS TACKLE DATA WHICH IS NOT VERY HELPFUL FOR OUR CASE
# WE SHOULD DEFINITELY CHECK TO SEE IF WE CAN FIND MORE DATA FOR OTHER SEASONS IF POSSIBLE

plays = plays.rename(columns={'ballCarrierId' : 'nflId'}) #rename attributes to their counterparts in the other files to enable join  

combined_df = pd.merge(plays, players, on='nflId', how='left') #join to get detailed player info 
combined_df = pd.merge(combined_df, games, on='gameId', how='left'); #join to get detailed game info

In [42]:
# conduct feature engineering to build the attributes we desire

#conversion of defensiveTeam to a quantitative variable: defensiveTeamQt 
#ref: https://stackoverflow.com/questions/38088652/pandas-convert-categories-to-numbers 
combined_df['defensiveTeamQt'] = pd.factorize(combined_df['defensiveTeam'], sort=True)[0] + 1; 

#convert offensiveFormation to a quantitative variable: offensiveFormationQt 
combined_df['offenseFormationQt'] = pd.factorize(combined_df['offenseFormation'], sort=True)[0] + 1; 
#display(pd.factorize(combined_df['offenseFormation'], sort=True)[1]); 

#conversion of gameClock to purely seconds remaining in the game
#ref: https://stackoverflow.com/questions/50308629/python-pandas-column-convert-minutes-to-second
combined_df['gameClockInSeconds'] = [(int(min) * 60) + int(sec) for min, sec in combined_df['gameClock'].str.split(':')]; 

#convert scoreDifferential and calculate ageOfBallCarrier
scoreDiff = []; 
ballCarrierAge = []; 
secondary = [] 
for i, j in combined_df.iterrows(): 
    #convert scoreDifferential into negative if down a certain number of points, or positive if up a certain number of points
    if j['possessionTeam'] == j['visitorTeamAbbr']: 
        scoreDiff.append(j['preSnapVisitorScore'] - j['preSnapHomeScore']); 
    else: 
        scoreDiff.append(j['preSnapHomeScore'] - j['preSnapVisitorScore']);

    #calculate ageOfBallCarrier
    if j['birthDate'] == '':
        ballCarrierAge.append('bullshit'); 
    else: 
        birthDate = pd.to_datetime(j['birthDate']); 
        gameDate = pd.to_datetime(j['gameDate']); 
        
        ballCarrierAge.append(gameDate.year - birthDate.year - ((gameDate.month, gameDate.day) < (birthDate.month, birthDate.day)))

#convert height into cm
combined_df['heightInCm'] = [((int(ft) * 30.48) + (int(inches) * 2.54)) for ft, inches in combined_df['height'].str.split('-')]; 

#convert weight into kgs
combined_df['weightInKg'] = [(int(weight) * 0.453592) for weight in combined_df['weight']]; 

#add scoreDifferential and ageOfBallCarrier into the combined dataset      
combined_df['signedScoreDiff'] = scoreDiff; 
combined_df['ballCarrierAge'] = ballCarrierAge; 

#round everything to two decimal places 
combined_df = combined_df.round(2); 
combined_df['heightInCm'] = combined_df['heightInCm'].round(1)

In [45]:
# select only the desired attributes we need and discard the rest
final_df = combined_df[['gameDate', 'possessionTeam', 'defensiveTeam', 'defensiveTeamQt', 'ballCarrierDisplayName', 'heightInCm', 'weightInKg', 'position', 'ballCarrierAge', 'quarter', 'down', 
                        'gameClock', 'gameClockInSeconds', 'yardsToGo', 'preSnapHomeScore', 'preSnapVisitorScore', 'signedScoreDiff', 'playResult', 
                        'absoluteYardlineNumber', 'offenseFormation', 'offenseFormationQt', 'defendersInTheBox']]; 

In [ ]:
# export the finalized dataset to a csv